## Module 2.3: Working with LSTMs in Keras (A Review)

We turn to implementing a type of recurrent neural network know as LSTM in the Keras functional API. In this module we will pay attention to:

1. Using the Keras functional API for defining models.
2. Mounting your Google drive to your Colab environment for file interface.
3. Generating synthetic data from a LSTM and sequence seed.

Those students who are comfortable with all these matters might consider skipping ahead.

Note that we will not spend time tuning hyper-parameters: The purpose is to show how different techniques can be implemented in Keras, not to solve particular data science problems as optimally as possible. Obviously, most techniques include hyper-parameters that need to be tuned for optimal performance.

First we import required libraries.

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Apr 20 09:52:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import sys
import numpy

from google.colab import drive

from keras.models import Sequential
from keras import Model
from keras.optimizers import Adadelta
from keras.layers import Dense,Dropout,LSTM,Input
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

We will have a little fun and try to teach a neural network to write like Lewis Carroll, the author of Alice in Wonderland.

Note, though, that the same technique can be used to model any sequential system, and generate simulations from seeds for such a system. Here the sequence are the characters written by Carroll during Alice in Wonderland, but it could be, for example, an industrial system that evolves in time. In that case, when we generate simulations of the system based on current and recent conditions we simulate the expected evolution of the system - something of great value!

We will use the [Project Gutenburg text file of Alice in Wonderland](https://www.gutenberg.org/files/11/11.txt). But we need to get the file into our colab environment and this takes some work.

First, you need to place the file in your google drive. We will assume that you will place it in a folder called "Mastering Keras Datasets", and that you rename it "Alice.txt". If you don't, you will need to the file path used in the code.

Once you have done that, you will need to mount your google drive in Colab. Run the following code and complete the required authorizations.

Note that you will need to mount your drive every time you use code from this tutorial.

In [3]:
# Note: You will need to mount your drive every time you 
# run code in this tutorial.
drive.mount('/content/drive')

Mounted at /content/drive


Now we can load the file using code and prepare the data. We want to work with sequences of 100 characters as input data, and our target will be the next (101st) character.

To keep things simple, we will ignore upper/lower case character distinctions, and cast all alphabetical characters to lower case. To allow our model to work with these characters, we will encode them as integers. We will then normalize them to real numbers between 0 and 1 and add a dimension (we are working with a system with a single feature). Finally we will one-hot encode the target character (see previous module for discussion of one-hot encoding). This is not the only way to handle the data, but it is a simple one.

We will also return the unnormalized and non-reshaped X data, the number of characters found and an integer coding to character dictionary, all for use later.


In [4]:
def load_alice (
    rawTextFile="/content/drive/My Drive/Mastering Keras/Alice.txt"   
    ):
    # load ascii text and covert to lowercase
    raw_text = open(rawTextFile, encoding='utf-8').read()
    raw_text = raw_text.lower()
    # create mapping of unique chars to integers
    chars = sorted(list(set(raw_text)))
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    int_to_char = dict((i, c) for i, c in enumerate(chars))
    # summarize the loaded data
    n_chars = len(raw_text)
    n_vocab = len(chars)
    print ("Total Characters: ", n_chars)
    print ("Total Vocab: ", n_vocab)
    # prepare the dataset of input to output pairs encoded as integers
    seq_length = 100
    dataX = []
    dataY = []
    for i in range(0, n_chars - seq_length, 1):
    	seq_in = raw_text[i:i + seq_length]
    	seq_out = raw_text[i + seq_length]
    	dataX.append([char_to_int[char] for char in seq_in])
    	dataY.append(char_to_int[seq_out])
    n_patterns = len(dataX)
    print ("Total Patterns: ", n_patterns)
    # reshape X to be [samples, time steps, features]
    X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
    # normalize
    X = X / float(n_vocab)
    # one hot encode the output variable
    Y = np_utils.to_categorical(dataY)
    return X,Y,dataX,n_vocab,int_to_char

Now lets load the data. X and Y are the input and target label datasets we will use in training. X_ is the un-reshaped X data for use later.

In [5]:
X,Y,X_,n_vocab,int_to_char = load_alice()

Total Characters:  164047
Total Vocab:  64
Total Patterns:  163947


You can play around below to look at the shape of the resulting X and Y arrays, as well as their contents. But they are no longer understandable character strings.

In [ ]:
# Play around here to look at data characteristics

Now we define our LSTM using the Keras function API. We are going to make use of LSTM layers, and add a dropout layer for regularization.

We will pass the data to the model defining function so that we can read input and output dimensions of it, rather than hard coding them.

For comparison, a second version of the function is included showing how to use the sequential approach.

In [6]:
def get_model (X,Y):
    # define the LSTM model
    inputs=Input(shape=(X.shape[1],X.shape[2]),name="Input")
    lstm1=LSTM(256, input_shape=(100,1),return_sequences=True)(inputs)
    drop1=Dropout(0.2)(lstm1)
    lstm2=LSTM(256)(drop1)
    drop2=Dropout(0.2)(lstm2)
    outputs=Dense(Y.shape[1], activation='softmax')(drop2)
    model=Model(inputs=inputs,outputs=outputs)
    return model

def get_model_sequential (X,Y):
    # define the LSTM model
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2]),return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(Y.shape[1], activation='softmax'))
    return model

We get our model.

In [7]:
model=get_model(X,Y)

Now we will define an optimizer and compile it. If you are unfamiliar with the different types of optimizers available in keras, I suggest you read the keras documentation [here](https://keras.io/optimizers/) and play around training the model with different alternatives.

In [8]:
opt=Adadelta()


And we compile our model with the optimizer ready for training. We use categorical crossentropy as our loss function as this is a good default choice for working with a multi-class categorical target variable (i.e. the next character labels).

In [9]:
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Now we will make a function to fit the model. We will not do this very professionally (it is just a fun project), and so will not use any validation data. Rather, we will just run the training for a number of epoches - by default 20, though you can change this.

We will, though, use a ModelCheckpoint callback to save the best performing weights and load these into the model and the conclusion of the training. Note that training performance should normally improve with more epoches, so this is unlikely to improve performance. What we really want is to be able to load the best weights without having to redo the training process (see below)

If you want to, you are encouraged to alter the code in this tutorial to work with a training and validation set, and adjust the fit function below to incorporate an EarlyStopping callback based on performance on the validation data.

We have two one LSTM layer, we are dealing with sequences of length 100. So if we 'unroll' it, we have a network of 200 LSTM layers. And inside these layers are infact multiple internal layers setting up the LSTM architecture! So this is actually a pretty big network, and training will take some time (about 200 hours on the free Colab environment for 200 epochs). This is probably too much to conveniently run yourself.

Here we have an example of how we could train it on Colab. Colab will eventually time out. The best thing to do is to save our weights file to our google drive, so we can load it at leisure later and resume training. This is what we will do. Remember that if you didn't use the default name for your folder in your google drive you should change the path string in the code.

In real life, you will also often want to save the state of the optimizer (so that it keeps its current learning rate, etc). You can do this by accessing and saving model.optimizer.get_state(). It is left as an exercise to implement this.

*It is not expected that you train the network using this function - see below to load trained weights from your google drive.*

In [10]:
def fit_model (model,X,Y,epochs=100):
    # define the checkpoint callback
    filepath="/content/drive/My Drive/Mastering Keras/alice_best_weights.hdf5" 
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, 
                                 save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    # fit the model
    model.fit(X, Y, epochs=epochs, batch_size=128, callbacks=callbacks_list)
    # load the best weights
    model.load_weights(filename)
    # return the final model
    return model


We would then fit (train) the model by calling the above function.

*It is not expected that you train the network using this function - see below to load trained weights from your google drive.*

In [11]:
model=fit_model(model,X,Y,100)

Epoch 1/100
1281/1281 [==============================] - 44s 20ms/step - loss: 4.1304 - accuracy: 0.0465

Epoch 00001: loss improved from inf to 4.11652, saving model to /content/drive/My Drive/Mastering Keras/alice_best_weights.hdf5
Epoch 2/100
1281/1281 [==============================] - 26s 20ms/step - loss: 4.0462 - accuracy: 0.1652

Epoch 00002: loss improved from 4.11652 to 3.97929, saving model to /content/drive/My Drive/Mastering Keras/alice_best_weights.hdf5
Epoch 3/100
1281/1281 [==============================] - 26s 20ms/step - loss: 3.5387 - accuracy: 0.1664

Epoch 00003: loss improved from 3.97929 to 3.43717, saving model to /content/drive/My Drive/Mastering Keras/alice_best_weights.hdf5
Epoch 4/100
1281/1281 [==============================] - 26s 21ms/step - loss: 3.2842 - accuracy: 0.1676

Epoch 00004: loss improved from 3.43717 to 3.26262, saving model to /content/drive/My Drive/Mastering Keras/alice_best_weights.hdf5
Epoch 5/100
1281/1281 [=============================

NameError: ignored

Here we will load saved weights. You can use the "alice_best_weights.hdf5" file that comes with the course - just place it in the same folder as the "alice.txt" file in your google drive. This file has been trained for 200 epoches, and gets a loss around 1.16.

If you train the network yourself, the best weights will be saved as "alice_best_weights.hdf5" in the same location as above. You can therefore use the same code in both cases.

In all cases remember to change the filepath if you are not using the default folder name.

If you are resuming this tutorial here in a new session, you should re-mount your Google drive using the earlier code, re-load the data, and then run this code block to load the weights into a new model. 

If you want to train the model further, you will need to compile it with an optimizer.

In [ ]:
model=get_model(X,Y)
filepath="/content/drive/My Drive/Mastering Keras Datasets/alice_best_weights.hdf5"
model.load_weights(filepath)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Now we can see if our network has mastered the art of writing like Lewis Carroll! Let's write a function to let us see, and then call it.

In [ ]:
def write_like_Lewis_Carroll(model,X_,n_vocab,int_to_char):
  # pick a random seed...
  start = numpy.random.randint(0, len(X_)-1)
  # ... in order to decide which X datum to use to start
  pattern = X_[start]

  print ("Seed:")
  print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
  # generate characters
  for i in range(1000):
    # We transform the integer mapping of the characters to
    # real numbers suitable for input into our model.
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocab)
    # We use the model to estimate the probability distribution for
    # the next character
    prediction = model.predict(x, verbose=0)
    # We choose as the next character whichever the model thinks is most likely
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    # We add the integer to our pattern... 
    pattern.append(index)
    # ... and drop the earliest integer from our pattern.
    pattern = pattern[1:len(pattern)]
  print ("\nDone.")

In [ ]:
write_like_Lewis_Carroll(model,X_,n_vocab,int_to_char)

Seed:
" for it to speak with.

alice waited till the eyes appeared, and then nodded. 'it's no use
speaking t "
o see the mock turtle shat '

'i should hiv tereat ' thought alice, 'i must be giederen seams to be a bonk,' she said to herself, 'it would be of very curious to onow what there was a sery dortut, and the ooral of that iss thin the cook and a large rister sha thought the was now one of the court.
but the dould not heve a little botrle of the thate with a things of tee the door, she could not hear the conlers on the coor with pisted so see it was she same sotnd and mook up and was that it was ouer the whnle shoiek, and the thought the was now a bot of ceain, and was domencd it voice and bookdrs shat the was nuire silent for a minute, and she was nooiing at the court.

'i should hit tere things,' said the caterpillar.

'well, perhaps you may bean the same siings tuertion,' the duchess said to the gryphon.

'what i cen the thing,' said the caterpillar.

'well, perhaps you may bea

If you run the above a few times, you will see that we have had some success - though we are still a long way from a good Alice in Wonderland simulator!

Here is an extract from one simulation I ran:

*'i should hit tere things,' said the caterpillar.*

*'well, perhaps you may bean the same siings tuertion,' the duchess said to the gryphon.*

*'what i cen the thing,' said the caterpillar.*

*'well, perhaps you may bean the same siings tuertion,' the mock turtle seplied,*

*'that i man the mice,' said the caterpillar.*

We have got to the point of basic sentence structure, quotations for speech, plausible characters given the context, etc. There remains misspellings, and occasional punctuation errors, and other issues. (And this was a good selection.) 

In fact, you should be able to do much better. Trying with 500 time points (predicting the 501st character from the preceeding 500) and using a three layer LSTM will lead to major improvements. So would using more training data (multiple Lewis Carole books). You can see the performance achieved on a Shakespeare simulator [here](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). 

If you have time, consider it an exercise to try to improve this implementation to that level - but be warned, the suggested changes would lead to training time being about 7 times longer for the same number of epochs, and of course more epoches would be required as it would be a more complex model. Since it would have taken 100+ hours on the Colab environment (which disconnects after a time limit) this is really only an exercise for those with access to a powerful local environment. 